In [1]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17533, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 17533 (delta 16), reused 0 (delta 0), pack-reused 17502 (from 5)
Receiving objects: 100% (17533/17533), 16.61 MiB | 24.55 MiB/s, done.
Resolving deltas: 100% (12005/12005), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.2 MB/s et

In [2]:
# Installing huggingface_hub to download from HF repo
!pip install huggingface_hub

from huggingface_hub import hf_hub_download

# Downloading the best.pt from 'niki-stha/asl-detection-yolov5'
model_path = hf_hub_download(repo_id="niki-stha/asl-detection-yolov5", filename="best.pt")

print("Model downloaded to:", model_path)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


best.pt:   0%|          | 0.00/14.9M [00:00<?, ?B/s]

Model downloaded to: /root/.cache/huggingface/hub/models--niki-stha--asl-detection-yolov5/snapshots/9459dc7beba324f5b29414eaaf465161f9a4b69a/best.pt


In [3]:
import os
import torch
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob
from sklearn.metrics import classification_report, confusion_matrix
from collections import defaultdict


model_path = hf_hub_download(repo_id="niki-stha/asl-detection-yolov5", filename="best.pt")
print(f"Model downloaded to: {model_path}")



test_base_path = '/content/drive/MyDrive/Test_Alphabet'


#  Loading the YOLOv5 model using the downloaded weights
model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path, force_reload=True)

#  Suppress AMP warnings and future warnings
warnings.filterwarnings("ignore", category=UserWarning, message="`torch.cuda.amp.autocast.*` is deprecated")
warnings.filterwarnings("ignore", category=FutureWarning)

model.conf = 0.60  # confidence threshold



#  A-Z letters
letters = [chr(c) for c in range(ord('A'), ord('Z') + 1)]

#  Allowed image formats
image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.gif'}

#  Tracking
total_images = 0
total_detected = 0
per_letter_stats = defaultdict(lambda: {'total': 0, 'detected': 0})

# For evaluation
y_true = []
y_pred = []

# 🚀 Inference loop
for letter in letters:
    folder_path = os.path.join(test_base_path, letter)
    if not os.path.exists(folder_path):
        print(f"Folder for letter {letter} not found, skipping.")
        continue

    img_files = glob(os.path.join(folder_path, '*'))
    total = 0
    detected_count = 0

    print(f"\nProcessing letter '{letter}' with {len(img_files)} files...")

    for img_path in img_files:
        ext = os.path.splitext(img_path)[1].lower()
        if ext not in image_extensions:
            print(f"Skipping non-image file: {img_path}")
            continue

        total += 1
        total_images += 1

        results = model(img_path)

        if len(results.xyxy[0]) > 0:
            # Pick the most confident detection
            pred_class = int(results.pred[0][0][-1].item())
            class_name = model.names[pred_class]
            y_pred.append(class_name)
            total_detected += 1
            detected_count += 1
        else:
            y_pred.append('None')  # if nothing detected

        y_true.append(letter)

    per_letter_stats[letter]['total'] = total
    per_letter_stats[letter]['detected'] = detected_count

    accuracy = detected_count / total * 100 if total > 0 else 0
    print(f"Letter '{letter}': {detected_count}/{total} detected ({accuracy:.2f}%)")

#  Overall Accuracy
overall_accuracy = total_detected / total_images * 100 if total_images > 0 else 0

print("\n=== Evaluation Summary ===")
print(f"Total images evaluated: {total_images}")
print(f"Total detections: {total_detected}")
print(f"✅ Overall Detection Accuracy: {overall_accuracy:.2f}%")

#  Classification Report
report = classification_report(y_true, y_pred, labels=letters, output_dict=True, zero_division=0)
report_df = pd.DataFrame(report).transpose()
report_csv_path = "/content/yolo_letter_classification_report.csv"
report_df.to_csv(report_csv_path)
print(f"\n📄 Classification report saved to: {report_csv_path}")

#  Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred, labels=letters)
conf_matrix_df = pd.DataFrame(conf_matrix, index=letters, columns=letters)
conf_matrix_path = "/content/yolo_letter_confusion_matrix.csv"
conf_matrix_df.to_csv(conf_matrix_path)
print(f"📄 Confusion matrix saved to: {conf_matrix_path}")

#  Display Confusion Matrix Heatmap
plt.figure(figsize=(18, 14))
sns.heatmap(conf_matrix_df, annot=True, fmt="d", cmap="YlGnBu", linewidths=0.5)
plt.title("YOLOv5 Letter Classification - Confusion Matrix", fontsize=16)
plt.ylabel("True Label", fontsize=14)
plt.xlabel("Predicted Label", fontsize=14)
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig("/content/yolo_letter_confusion_matrix_heatmap.png")
plt.show()


Model downloaded to: /root/.cache/huggingface/hub/models--niki-stha--asl-detection-yolov5/snapshots/9459dc7beba324f5b29414eaaf465161f9a4b69a/best.pt


/usr/local/lib/python3.11/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 2025-8-18 Python-3.11.13 torch-2.6.0+cu124 CPU

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7313943 parameters, 0 gradients
Adding AutoShape... 



Processing letter 'A' with 100 files...
Letter 'A': 99/100 detected (99.00%)

Processing letter 'B' with 100 files...
Letter 'B': 87/100 detected (87.00%)

Processing letter 'C' with 100 files...
Letter 'C': 95/100 detected (95.00%)

Processing letter 'D' with 100 files...
Letter 'D': 75/100 detected (75.00%)

Processing letter 'E' with 100 files...
Letter 'E': 99/100 detected (99.00%)

Processing letter 'F' with 100 files...
Letter 'F': 95/100 detected (95.00%)

Processing letter 'G' with 100 files...
Letter 'G': 72/100 detected (72.00%)

Processing letter 'H' with 100 files...
Letter 'H': 79/100 detected (79.00%)

Processing letter 'I' with 100 files...
Letter 'I': 84/100 detected (84.00%)

Processing letter 'J' with 100 files...
Letter 'J': 99/100 detected (99.00%)

Processing letter 'K' with 100 files...
Letter 'K': 78/100 detected (78.00%)

Processing letter 'L' with 100 files...
Letter 'L': 95/100 detected (95.00%)

Processing letter 'M' with 100 files...
Letter 'M': 86/100 dete

In [ ]:

import os
import warnings
from glob import glob
from collections import defaultdict

import torch
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    precision_recall_fscore_support,
)

from huggingface_hub import hf_hub_download




# Setting Path to Testing Dataset
test_base_path = "/content/drive/MyDrive/Test_Alphabet"

# Confidence threshold for YOLO detections
CONF_THRESH = 0.60

# Evaluation Output paths
report_csv_path = "/content/yolo_letter_classification_report.csv"
conf_matrix_path = "/content/yolo_letter_confusion_matrix.csv"
conf_matrix_png_path = "/content/yolo_letter_confusion_matrix_heatmap.png"
metrics_summary_path = "/content/yolo_letter_metrics_summary.csv"


#  Download weights from HF Hub

model_path = hf_hub_download(
    repo_id="niki-stha/asl-detection-yolov5",
    filename="best.pt"
)
print(f"Model downloaded to: {model_path}")


# Loading the model

# Suppress specific warnings
warnings.filterwarnings("ignore", category=UserWarning, message="`torch.cuda.amp.autocast.*` is deprecated")
warnings.filterwarnings("ignore", category=FutureWarning)

# Load YOLOv5 custom model
model = torch.hub.load("ultralytics/yolov5", "custom", path=model_path, force_reload=True)
model.conf = CONF_THRESH  # confidence threshold


#  Labels & bookkeeping

letters = [chr(c) for c in range(ord("A"), ord("Z") + 1)]
image_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".gif"}

total_images = 0
total_detected = 0
per_letter_stats = defaultdict(lambda: {"total": 0, "detected": 0})

y_true = []
y_pred = []


#  Inference loop

for letter in letters:
    folder_path = os.path.join(test_base_path, letter)
    if not os.path.exists(folder_path):
        print(f"Folder for letter {letter} not found, skipping.")
        continue

    img_files = glob(os.path.join(folder_path, "*"))
    total = 0
    detected_count = 0

    print(f"\nProcessing letter '{letter}' with {len(img_files)} files...")

    for img_path in img_files:
        ext = os.path.splitext(img_path)[1].lower()
        if ext not in image_extensions:
            # Skip non-image files
            continue

        total += 1
        total_images += 1

        # Run inference
        results = model(img_path)

        # YOLOv5 returns detections as [x1, y1, x2, y2, conf, cls] in results.xyxy[0]
        det = results.xyxy[0]
        if det is not None and len(det) > 0:
            # pick the most confident detection

            top_idx = det[:, 4].argmax().item()
            pred_class_idx = int(det[top_idx, 5].item())
            class_name = str(model.names[pred_class_idx])

            y_pred.append(class_name)
            total_detected += 1
            detected_count += 1
        else:
            # nothing detected for this image
            y_pred.append("None")

        y_true.append(letter)

    per_letter_stats[letter]["total"] = total
    per_letter_stats[letter]["detected"] = detected_count

    accuracy_letter = (detected_count / total * 100) if total > 0 else 0.0
    print(f"Letter '{letter}': {detected_count}/{total} detected ({accuracy_letter:.2f}%)")


#  Overall detection accuracy

overall_detection_accuracy = (total_detected / total_images * 100) if total_images > 0 else 0.0

print("\n=== Evaluation Summary ===")
print(f"Total images evaluated: {total_images}")
print(f"Total detections: {total_detected}")
print(f"✅ Overall Detection Accuracy: {overall_detection_accuracy:.2f}%")


#  Classification metrics (A–Z only)

# Classification Accuracy
clf_accuracy = accuracy_score(y_true, y_pred)

# Macro-averaged Precision/Recall/F1 across A–Z (treat each class equally)
macro_p, macro_r, macro_f1, _ = precision_recall_fscore_support(
    y_true,
    y_pred,
    labels=letters,        # restrict to A–Z
    average="macro",
    zero_division=0
)

print("\n=== Classification Metrics (A–Z) ===")
print(f"🎯 Classification Accuracy: {clf_accuracy*100:.2f}%")
print(f"📌 Macro Precision: {macro_p:.4f}")
print(f"📌 Macro Recall:    {macro_r:.4f}")
print(f"📌 Macro F1-Score:  {macro_f1:.4f}")

# Save compact metrics summary
pd.DataFrame(
    {
        "metric": ["classification_accuracy", "macro_precision", "macro_recall", "macro_f1", "overall_detection_accuracy"],
        "value":  [clf_accuracy,              macro_p,           macro_r,       macro_f1,   overall_detection_accuracy/100.0],
    }
).to_csv(metrics_summary_path, index=False)
print(f"📄 Metrics summary saved to: {metrics_summary_path}")


#  Classification report (per-class) — A–Z

report = classification_report(
    y_true,
    y_pred,
    labels=letters,
    output_dict=True,
    zero_division=0
)
report_df = pd.DataFrame(report).transpose()
report_df.to_csv(report_csv_path)
print(f"📄 Classification report saved to: {report_csv_path}")


#  Confusion Matrix (A–Z)

conf_matrix = confusion_matrix(y_true, y_pred, labels=letters)
conf_matrix_df = pd.DataFrame(conf_matrix, index=letters, columns=letters)
conf_matrix_df.to_csv(conf_matrix_path)
print(f"📄 Confusion matrix saved to: {conf_matrix_path}")


#  Confusion Matrix Heatmap

plt.figure(figsize=(18, 14))
sns.heatmap(conf_matrix_df, annot=True, fmt="d", cmap="YlGnBu", linewidths=0.5)
plt.title("YOLOv5 Letter Classification - Confusion Matrix", fontsize=16)
plt.ylabel("True Label", fontsize=14)
plt.xlabel("Predicted Label", fontsize=14)
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig(conf_matrix_png_path, dpi=150)
print(f"🖼️ Confusion matrix heatmap saved to: {conf_matrix_png_path}")
plt.show()


Model downloaded to: /root/.cache/huggingface/hub/models--niki-stha--asl-detection-yolov5/snapshots/9459dc7beba324f5b29414eaaf465161f9a4b69a/best.pt


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2025-8-16 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7313943 parameters, 0 gradients
Adding AutoShape... 



Processing letter 'A' with 100 files...
Letter 'A': 99/100 detected (99.00%)

Processing letter 'B' with 100 files...
Letter 'B': 87/100 detected (87.00%)

Processing letter 'C' with 100 files...
Letter 'C': 95/100 detected (95.00%)

Processing letter 'D' with 100 files...
Letter 'D': 75/100 detected (75.00%)

Processing letter 'E' with 100 files...
Letter 'E': 99/100 detected (99.00%)

Processing letter 'F' with 100 files...
Letter 'F': 95/100 detected (95.00%)

Processing letter 'G' with 100 files...
Letter 'G': 72/100 detected (72.00%)

Processing letter 'H' with 100 files...
Letter 'H': 79/100 detected (79.00%)

Processing letter 'I' with 100 files...
Letter 'I': 84/100 detected (84.00%)

Processing letter 'J' with 100 files...
Letter 'J': 99/100 detected (99.00%)

Processing letter 'K' with 100 files...
Letter 'K': 78/100 detected (78.00%)

Processing letter 'L' with 100 files...
Letter 'L': 95/100 detected (95.00%)

Processing letter 'M' with 100 files...
Letter 'M': 86/100 dete